In [ ]:
import cv2
from ultralytics import YOLO
import time
import math
# import torch # Import the torch library


# --- Main Settings ---
WEBCAM_INDEX = 0  # Default webcam is 0
CONFIDENCE_THRESHOLD = 0.5 # Initial confidence threshold
FONT = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.8
FONT_COLOR = (255, 255, 255) # White
BOX_COLOR = (0, 255, 0) # Green
BOX_THICKNESS = 2

def main():
    ########################################
    # # --- GPU VERIFICATION ---
    # # Check if a CUDA-enabled GPU is available and print the device name
    # if torch.cuda.is_available():
    #     device = torch.device("cuda")
    #     print(f"GPU is available. Using device: {torch.cuda.get_device_name(0)}")
    # else:
    #     device = torch.device("cpu")
    #     print("GPU not available. Using CPU.")
    ########################################


    # 1. Load the pretrained YOLOv8 model and move it to the selected device
    print("Loading model...")
    # The model will be automatically sent to the 'device' (GPU or CPU)
    model = YOLO("yolov8n.pt") # Using the nano version for complete performance
    # model.to(device) # Explicitly send the model to the GPU
    class_names = model.names
    print("Model loaded.")

    # 2. Initialize webcam capture
    cap = cv2.VideoCapture(WEBCAM_INDEX)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return

    # Variables for FPS calculation
    prev_frame_time = 0

    while True:
        # 3. Read a frame from the webcam
        success, frame = cap.read()
        if not success:
            print("Error: Failed to capture frame.")
            break

        # 4. Perform inference on the frame
        results = model(frame, stream=True)
        # # The model will process the frame on the GPU
        # results = model(frame, stream=True, verbose=False) # verbose=False cleans up console output

        # 5. Process and display detection results
        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Filter detections by confidence
                confidence = math.ceil(box.conf[0] * 100) / 100
                if confidence < CONFIDENCE_THRESHOLD:
                    continue

                # Get bounding box coordinates
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Get class name
                cls_index = int(box.cls[0])
                cls_name = class_names[cls_index]

                # Draw bounding box and label
                label = f"{cls_name}: {confidence}"
                # label = f"{cls_name}: {confidence:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), BOX_COLOR, BOX_THICKNESS)
                cv2.putText(frame, label, (x1, y1 - 10), FONT, FONT_SCALE, FONT_COLOR, BOX_THICKNESS)

        # 6. Calculate and display FPS
        new_frame_time = time.time()
        fps = 1 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), FONT, FONT_SCALE, FONT_COLOR, BOX_THICKNESS)
        ################
        # # To avoid division by zero error on the first frame
        # if prev_frame_time > 0:
        #     fps = 1 / (new_frame_time - prev_frame_time)
        #     # Display FPS in a prominent color like red
        #     cv2.putText(frame, f"FPS: {int(fps)}", (10, 30), FONT, FONT_SCALE, (0, 0, 255), BOX_THICKNESS)
        
        # prev_frame_time = new_frame_time
        ##############

        # 7. Display the final frame
        cv2.imshow("Real-Time Object Detection", frame)

        # 8. Exit loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # 9. Release resources
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()